In [1]:
'''
Для работа нашего генератора нам требуется понимание с каких ip-адресов на какие комьютеры (какие ip-адреса), 
с каких портов и на какие порты отправляются сетевые пакеты с данными как при атаках, так и при обычных 
пользовательских запросах. Это необходимо знать для того, чтобы сгенерировать пакеты с данными, которые 
будут приближены к данным в том датасете, на котором Вы обучаете свою модель машинного обучения и 
нейросеть для детектирования атак

Этот файл посвящен тому, чтобы вычленить из нашего ip-адреса машин с которых отправляются данные, 
сгруппировать их по признаку боты это или обычные пользовательские комьютеры. И для каждой такой машины
агрегировать список ip-адресов на которые она отправляла запросы, список портов с которых отправлялись
сетевые пакеты, список портов на которые отправлялись пакеты, среднее время отправки пакетов данных,
средняя длина пакетов, среднее количество пакетов во время одной сессии и т.п. Эти данные позволят нам
генерировать похожие пакеты для найденных ip-адресов комьютеров
'''
# импортируем библиотеки pandas и numpy для работы с табличными данными
import pandas as pd
import numpy as np

In [11]:
'''
Для примера был взят файл с https://www.kaggle.com/devendra416/ddos-datasets. Был взят датасет из папки
ddos_balanced. Т.к. размер файла с данными очень большой (более 6 ГБ). Приходится применять специальные механизмы
для загрузки данных. Для этого:
1. Файл загружался блоками (chunks) по 400 тыс. строк в каждом блоке (значение переменной size)
2. При загружке данных они сразу сжимались, путем группировки данных по ip-адресу машины источника данных 
(Поле Src IP), вычислялись средние числовые показатели, а так же группировались во множества (set) нечисловые
(категориальные) параметры (порт источника, порт назначения, ip-адрес назначения и др.). Т.к. в python множество - 
это структура данных, которая содержит только уникальные элементы, то при группировке данных значений во множество,
остаются только уникальные значения этих параметров в рамках данного блока (chunk) данных из большого файла, т.е.
для одного ip-адреса в поле Dst IP будут храниться только уникальные ip-адреса, для Protocol - только уникальные
номера протоколов по которым отправлял данные этот комьютер и т.д. Это значительно сокращает объем получаемых данных
'''
# загружаем файл блоками по 400 тыс. строк
size = 4 * 10 ** 4
readerCSV = pd.read_csv("dataset/ddos_balanced/final_dataset.csv", sep=",", chunksize=size)

# каждый блок таких данных будет храниться в этом списке
ddos_result = []
i = 0
iter_count = 12794627 / size
# data = next(readerCSV)
for data in readerCSV:
    # заполняем пропуски нулями - полезно для расчета средних значений непрерывных переменных
    data = data.fillna(0)
    # группируем данные по ip-адресу источника пакетов и вычисляем средние показатели по всем числовы параметром
    # из общего числа параметров были выбраны только некоторые, которые играют важную роль при формировании пакетов
    # и помещатся в памяти при загрузке. Полный перечень данных в памяти не поместился

    flows_count = data.groupby(['Src IP', 'Dst IP']).agg({'Flow ID': 'count'})

    mean_flows_count = flows_count.groupby(['Src IP']).mean().rename(columns={'Flow ID': 'Flows Cnt Mean'})

    std_flows_count = flows_count.groupby(['Src IP']).std().rename(columns={'Flow ID': 'Flows Cnt Std'})

    cnt_data = data.groupby(['Src IP']).mean()[['Tot Fwd Pkts', 'Tot Bwd Pkts', 'Fwd Pkt Len Std', 
                                                'Fwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Bwd Pkt Len Mean', 'Flow IAT Mean', 
                                                'Flow IAT Std', 'Fwd PSH Flags', 'Bwd PSH Flags', 'SYN Flag Cnt', 
                                                'FIN Flag Cnt', 'URG Flag Cnt', 'Flow Byts/s', 'Flow Pkts/s', 'Bwd Pkts/s', 
                                                'Down/Up Ratio', 'Idle Std', 'Idle Mean', 'Active Mean', 'Active Std']]
    # группируем данные по ip-адресу источника пакетов и агрегируем категориальные параметры во множества
    dst_ip_data = data.groupby(['Src IP']).agg({'Dst IP': set})
    protocol_data = data.groupby(['Src IP']).agg({'Protocol': set})
    dst_port_data = data.groupby(['Src IP']).agg({'Dst Port': set})
    src_ports_data = data.groupby(['Src IP']).agg({'Src Port': set})
    labels_data = data.groupby(['Src IP']).agg({'Label': set})
    duration_data = data.groupby(['Src IP']).agg({'Flow Duration': list})
    # std_flow_duration = data.groupby(['Src IP']).agg({'Flow Duration': 'std'}).rename(columns={'Flow Duration': 'Flow Duration Std'})
    # std_flow_duration

    # объединяем получившиеся отдельные наборы данных единую таблицу
    ddos_data = src_ports_data.join(dst_ip_data).join(dst_port_data).join(protocol_data) \
        .join(labels_data).join(duration_data).join(cnt_data).join(mean_flows_count).join(std_flows_count)

    # добавляем таблицу данных для данного блока из файла в общий список блоков
    ddos_result.append(ddos_data)

    # это просто вывод процента загрузки данных из файла. загрузка данных происходит минут 5-10 на моём комьютере. На
    # Вашем может быть больше или меньше
    i += 1
    percent = i / iter_count * 100
    print("Чтение данных: " + "{0:.2f}".format(percent if percent < 100 else 100) + "%", end="\r")

print()
print('finished')
readerCSV = None
# data = None
# data[data['Src IP'] == '192.168.1.101'][['Flow Duration', 'Tot Fwd Pkts', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt']]

data[['Flow Duration', 'Flow Pkts/s', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'Bwd Pkts/s']]

Чтение данных: 0.31%
finished


,Flow Duration,Flow Pkts/s,Tot Fwd Pkts,Tot Bwd Pkts,Bwd Pkts/s
0,3974862,18.365417,29,44,11.069567
1,63,31746.031746,1,1,15873.015873
2,476078,16.803969,2,6,12.602977
3,151,19867.549669,2,1,6622.516556
4,472507,14.814595,2,5,10.581854
...,...,...,...,...,...
39995,8999866,0.222226,1,1,0.111113
39996,5,400000.000000,0,2,400000.000000
39997,63113322,0.110912,6,1,0.015845
39998,16808910,0.118985,1,1,0.059492


In [ ]:
# соединяем отдельные блоки полученных данных из файла (предварительно уже преобразованных по правилам, которые описаны
# выше) в единую таблицу
intermediate_res = pd.concat(ddos_result, sort=False)
ddos_result = []
intermediate_res

In [13]:
'''
В связи с тем, что мы сгруппировали только данные внутри блоков и оставили уникальные ip-адреса источников данных только
внутри каждого блока (chunk) из файла - тем не менее между блоками ip-адреса так же могут пересекаться. Необходимо
теперь провести аналогичную группировку данных по всей собранной таблице, попутно вычислив средние показатели для
непрерывных переменных, и собрать получившиеся множества категориальных переменных в единый блок для каждого ip-адреса.
Т.е. из каждого блока (chunk) для конкретного ip-адреса собрать вместе все ip-адреса назначения, протоколы, порты источника,
порты назначения и т.п. Но сделать это простым методом не удастся. Т.к. в каждом блоке (chunk) данные были собраны во
множества - нет возмоности средствами pandas объединить множества при очередной агрегации (во всяком случае я его не
нашел). Поэтому мы объединяем множества в списки (получается список множеств) и позже его обработаем для большего удобства
'''
# вновь агрегируем данные по ip-адресу источника уже по единой таблице, вычисляя средние значения числовых параметров
cnt_data = intermediate_res.groupby(['Src IP']).mean()[['Tot Fwd Pkts', 'Tot Bwd Pkts', 'Fwd Pkt Len Std', 
                                                'Fwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Bwd Pkt Len Mean', 'Flow IAT Mean', 
                                                'Flow IAT Std', 'Fwd PSH Flags', 'Bwd PSH Flags', 'SYN Flag Cnt', 
                                                'FIN Flag Cnt', 'URG Flag Cnt', 'Flow Byts/s', 'Flow Pkts/s', 'Bwd Pkts/s', 
                                                'Down/Up Ratio', 'Idle Std', 'Idle Mean', 'Active Mean', 'Active Std',
                                                'Flows Cnt Mean', 'Flows Cnt Std']]

# вновь агрегируем данные по ip-адресу источника уже по единой таблице, объединяя множества категориальных переменных
# в списки множеств (как описано выше)
dst_ip_data = intermediate_res.groupby(['Src IP'])['Dst IP'].apply(np.hstack).to_frame()
protocol_data = intermediate_res.groupby(['Src IP'])['Protocol'].apply(np.hstack).to_frame()
dst_port_data = intermediate_res.groupby(['Src IP'])['Dst Port'].apply(np.hstack).to_frame()
src_ports_data = intermediate_res.groupby(['Src IP'])['Src Port'].apply(np.hstack).to_frame()
labels_data = intermediate_res.groupby(['Src IP'])['Label'].apply(np.hstack).to_frame()
duration_data = intermediate_res.groupby(['Src IP'])['Flow Duration'].apply(np.hstack).to_frame()

# собираем отдельные блоки данных в единую таблицу
result_data = dst_ip_data.join(protocol_data).join(dst_port_data).join(src_ports_data).join(duration_data).join(labels_data).join(cnt_data)
result_data
# dst_ip_data

,Dst IP,Protocol,Dst Port,Src Port,Flow Duration,Label,Tot Fwd Pkts,Tot Bwd Pkts,Fwd Pkt Len Std,Fwd Pkt Len Mean,...,Flow Byts/s,Flow Pkts/s,Bwd Pkts/s,Down/Up Ratio,Idle Std,Idle Mean,Active Mean,Active Std,Flows Cnt Mean,Flows Cnt Std
Src IP,,,,,,,,,,,,,,,,,,,,,
192.168.1.101,"[{74.55.1.4, 97.74.104.201, 97.74.144.108}]",[{6}],"[{80, 443}]","[{4100, 2055, 2057, 2061, 2068, 2069, 2076, 20...","[6163053, 95002467, 1421816, 5687268, 5225668,...",[{ddos}],6.884336,8.625909,31.156098,24.178063,...,1135.376672,260.629267,127.428095,0.571051,5.361768e+04,6.644611e+06,8.336376e+04,3.530006e+04,504.333333,844.953450
192.168.1.102,"[{74.55.1.4, 97.74.104.201, 69.84.133.138}]",[{6}],[{80}],"[{3076, 3077, 3079, 3082, 3083, 3088, 3090, 30...","[119942533, 14722879, 119777479, 111202320, 10...",[{ddos}],84.918768,159.750700,15.526194,6.713555,...,16843.750534,42818.575821,21270.000309,0.980392,3.659573e+04,1.003268e+06,3.734518e+05,9.916368e+03,119.000000,171.717792
192.168.1.103,"[{69.192.24.88, 74.55.1.4, 97.74.144.108, 69.8...",[{6}],[{80}],"[{2049, 2563, 2052, 2053, 2054, 3075, 3258, 35...","[4134203, 5977626, 3040957, 632174, 3439332, 4...",[{ddos}],38.111709,62.118439,18.355966,9.952544,...,225837.053007,9638.299695,4802.931374,0.768506,3.716657e+04,1.711862e+06,5.116172e+05,6.658924e+04,148.600000,224.602538
192.168.1.104,"[{69.192.24.88, 74.55.1.4, 203.73.24.75, 97.74...",[{6}],"[{80, 443}]","[{21574, 18449, 18450, 18451, 20523, 20524, 20...","[6108487, 95010697, 1421818, 5577930, 216986, ...",[{ddos}],5.443097,9.551819,30.109324,19.114062,...,2976.592021,39439.540841,19220.034793,1.319124,0.000000e+00,3.850293e+05,1.631446e+04,0.000000e+00,426.166667,846.936460
192.168.1.105,[{97.74.104.201}],[{6}],"[{80, 443}]","[{17408, 18560, 17412, 17418, 18582, 18585, 18...","[79516370, 119531746, 113020493, 119974637, 89...",[{ddos}],71.464286,124.642857,21.366217,12.969670,...,3933.221991,167.372200,87.318461,0.583333,1.569676e+06,7.704307e+06,3.369959e+06,8.929075e+05,84.000000,NaN
192.168.2.108,"[{69.192.24.88, 74.55.1.4, 203.73.24.75, 97.74...",[{6}],[{80}],"[{4096, 2049, 4097, 4071, 4098, 2053, 2054, 40...","[6184740, 3655938, 5179480, 33180335, 28697973...",[{ddos}],16.215455,25.967758,26.560285,16.699158,...,1725.282864,1033.205350,455.368886,0.942170,1.766022e+05,1.907240e+06,3.914389e+05,3.209584e+05,390.800000,302.385846
192.168.2.109,"[{97.74.104.201, 203.73.24.75, 67.220.214.50}]",[{6}],"[{80, 443}]","[{1026, 1027, 1028, 1029, 1030, 1031, 1032, 10...","[912283, 141, 810661, 244, 39037231, 245, 1462...",[{ddos}],2.895106,4.901064,23.334377,15.191693,...,2162.291193,15999.565784,7709.606175,1.198298,0.000000e+00,8.872459e+04,8.889210e+03,0.000000e+00,3133.333333,5421.896470
192.168.2.110,"[{69.192.24.88, 67.220.214.50, 97.74.144.108, ...",[{6}],"[{80, 443}]","[{1026, 1027, 1028, 1029, 1030, 1031, 1032, 10...","[101250028, 333595, 171, 346561, 134, 370944, ...",[{ddos}],17.513307,32.840481,18.097868,9.598423,...,15002.644846,27299.719730,13571.294329,1.238685,1.720557e+04,4.623277e+05,2.527272e+04,9.394776e+02,1179.800000,1460.215635
192.168.2.111,"[{74.55.1.4, 208.113.162.153}]",[{6}],[{80}],"[{3712, 4069, 3687, 4045, 3726, 3982, 3702, 39...","[5145388, 2167236, 8937564, 937436, 5627553, 1...",[{ddos}],7.500000,12.892857,13.283336,7.957569,...,4298.296506,5.723639,3.612610,0.857143,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,14.000000,2.828427


In [14]:
'''
На этом этапе будем обрабатывать получившиеся списки множеств категориальных переменных. С такой структурой данных работать
довольно сложно, поэтому будем ее упрощать

В настоящий момент каждая строка result_data - это строка с данными для одного ip-адреса источника: список множеств 
ip-адресов комьютеров, на которые отправлялись сетевые пакеты, список множеств портов с которых отправлялись пакеты,
список множеств портов на которые отправлялись пакеты, среднее значение интервала обмена данными, среднее значение
длины пакета, среднее значение количества пакетов, отправлемых с данного комьютера во время одного сеанса и т.д.

Что из себя представляет структура "список множеств". Например, для строки 5 таблицы result_data значение её поля Dst IP
(ip-адрес, куда с источника отправлялись данные) имеет следующий вид:
[{172.31.67.62}, {172.31.67.62}, {172.31.67.62}]

Это список из 3х множеств в каждом из которых по одному элементу. При этом - элементы во множествах одинаковые. Логично
будет объединить все множества категориальных переменных, оставив только уникальные значения. В результате получим
следующий список ip-адресов, на которые отправлялись данные с источника: [172.31.67.62]

Единственная проблема - в датафрейм pandas нельза записывать список. Поэтому мы сделаем из элементов множеств - строки
и объединим их через пробел. Так для поля Dst IP получим строку из ip-адресов, разделенных пробелами, для поля Dst Port - 
строку из номеров портов, разделенных пробелами и т.д. Измененые данных категоримальных переменных выполняется на моем
комьютере минут 15-20. На Вашем может быть больше или меньше
'''
# перебираем все строки общей таблицы
for index, row in result_data.iterrows():
    #формируем пустое множество
    p = set()
    # берем поле категориальной переменной Dst IP. Это список. Получаем каждое множество из этого списка
    for d in row['Dst IP']:
        # и объединяем со множеством p
        p |= d
    # формируем из элементов итогового множества - строки и объединяем их через пробелы
    # результат записываем в нашу таблицу в исходное поле, откуда брали данные для преобразования
    result_data.loc[index,'Dst IP'] = " ".join(map(str, p))
    
    # выполняем описанные выше операции для всех категориальных переменных
    p = set()
    for d in row['Protocol']:
        p |= d
    result_data.loc[index,'Protocol'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Dst Port']:
        p |= d
    result_data.loc[index,'Dst Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Src Port']:
        p |= d
    result_data.loc[index,'Src Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Label']:
        p |= d
    result_data.loc[index,'Label'] = " ".join(map(str, p))

    result_data.loc[index, 'Flow Duration Mean'] = np.mean(row['Flow Duration'].tolist())
    result_data.loc[index, 'Flow Duration Std'] = np.std(row['Flow Duration'].tolist())

# смотрим на результат. Теперь можно получить для любого ip-адреса источника значение поля, например, Dst IP, разбить
# хранящуюся там строку по пробелам и получить список ip-адресов назначения, на которые отправлялись пакеты с 
# этого источника. И так для любой категориальной переменной
del result_data['Flow Duration']
result_data = result_data.fillna(0)
result_data

,Dst IP,Protocol,Dst Port,Src Port,Label,Tot Fwd Pkts,Tot Bwd Pkts,Fwd Pkt Len Std,Fwd Pkt Len Mean,Bwd Pkt Len Std,...,Bwd Pkts/s,Down/Up Ratio,Idle Std,Idle Mean,Active Mean,Active Std,Flows Cnt Mean,Flows Cnt Std,Flow Duration Mean,Flow Duration Std
Src IP,,,,,,,,,,,,,,,,,,,,,
192.168.1.101,74.55.1.4 97.74.104.201 97.74.144.108,6,80 443,4100 2055 2057 2061 2068 2069 2076 2087 2094 2...,ddos,6.884336,8.625909,31.156098,24.178063,274.969526,...,127.428095,0.571051,5.361768e+04,6.644611e+06,8.336376e+04,3.530006e+04,504.333333,844.953450,1.563599e+07,2.067458e+07
192.168.1.102,74.55.1.4 97.74.104.201 69.84.133.138,6,80,3076 3077 3079 3082 3083 3088 3090 3091 3602 3...,ddos,84.918768,159.750700,15.526194,6.713555,231.009040,...,21270.000309,0.980392,3.659573e+04,1.003268e+06,3.734518e+05,9.916368e+03,119.000000,171.717792,1.664876e+07,3.555730e+07
192.168.1.103,208.113.162.153 69.192.24.88 74.55.1.4 97.74.1...,6,80,2049 3075 2052 2053 2054 3083 3085 2066 2067 3...,ddos,38.111709,62.118439,18.355966,9.952544,197.983129,...,4802.931374,0.768506,3.716657e+04,1.711862e+06,5.116172e+05,6.658924e+04,148.600000,224.602538,2.002657e+07,3.160028e+07
192.168.1.104,67.220.214.50 203.73.24.75 69.192.24.88 97.74....,6,80 443,20523 20524 20525 20540 20544 20545 20546 2054...,ddos,5.443097,9.551819,30.109324,19.114062,277.920751,...,19220.034793,1.319124,0.000000e+00,3.850293e+05,1.631446e+04,0.000000e+00,426.166667,846.936460,6.761965e+06,2.069437e+07
192.168.1.105,97.74.104.201,6,80 443,17408 18560 17412 17418 18582 18585 18458 1794...,ddos,71.464286,124.642857,21.366217,12.969670,186.880747,...,87.318461,0.583333,1.569676e+06,7.704307e+06,3.369959e+06,8.929075e+05,84.000000,0.000000,5.341630e+07,4.669583e+07
192.168.2.108,203.73.24.75 69.192.24.88 74.55.1.4 97.74.144....,6,80,4096 2049 4097 4071 4098 2053 2054 4099 4100 4...,ddos,16.215455,25.967758,26.560285,16.699158,243.888516,...,455.368886,0.942170,1.766022e+05,1.907240e+06,3.914389e+05,3.209584e+05,390.800000,302.385846,1.881270e+07,3.144091e+07
192.168.2.109,97.74.104.201 203.73.24.75 67.220.214.50,6,80 443,1026 1027 1028 1029 1030 1031 1032 1033 1034 1...,ddos,2.895106,4.901064,23.334377,15.191693,218.240976,...,7709.606175,1.198298,0.000000e+00,8.872459e+04,8.889210e+03,0.000000e+00,3133.333333,5421.896470,8.122684e+05,2.377511e+06
192.168.2.110,97.74.104.201 69.192.24.88 97.74.144.108 67.22...,6,80 443,1026 1027 1028 1029 1030 1031 1032 1033 1034 1...,ddos,17.513307,32.840481,18.097868,9.598423,221.446437,...,13571.294329,1.238685,1.720557e+04,4.623277e+05,2.527272e+04,9.394776e+02,1179.800000,1460.215635,5.107447e+06,1.948278e+07
192.168.2.111,74.55.1.4 208.113.162.153,6,80,3712 4069 3687 4045 3726 3982 3702 3933 3678,ddos,7.500000,12.892857,13.283336,7.957569,167.968307,...,3.612610,0.857143,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,14.000000,2.828427,2.253677e+07,3.534104e+07


In [15]:
# отфильтруем строки таблицы по метке (ddos - источник отправлял пакеты для ddos атаки)
ddos_data = result_data[result_data['Label'] == 'ddos']
# запишем отфильтрованную таблицу в отдельный файлd dos_src.csv - можете его посмотреть, он не очень большой
ddos_data.to_csv("ddos_src.csv")
# посмотрим сколько же строк, т.е. ip-адресов в этой таблице. Оказывается, что ddos атаки в этом датасете производились
# всего с 34 комьютеров. Немного, я думал что сеть для атаки больше. Но это не важно. Главное что мы понимаем, для
# каких комьютеров генерировать данные ddos-атаки. И знаем все показатели пакетов атаки
len(ddos_data)

20

In [16]:
# отфильтруем строки таблицы по метке (Benign - источник отправлял пакеты для обычных пользвательских запросов)
benign_data = result_data[result_data['Label'] == 'Benign']
# запишем и эти данные в csv файл. Она значительно больше. Оно и понятно, мы взяли для примера не сбалансированный
# датасет, где данные, судя по описанию dataset распределены в соотношении 20% данных атак, 80% данных чистых запросов
benign_data.to_csv("benign_src.csv")
# в этом подмножестве таблицы оказалось 36724 адреса. Можем использовать эти данные для генерации пакетов чистых
# пользовательских запросов
len(benign_data)

0

In [17]:
'''
Теперь у нас есть все данных для геренации пакетов. Осталось научиться загружать их в наш генератор и использовать
для генерации пакетов. Работу продолжим в файле generator.py. Открывайте его
'''

'\nТеперь у нас есть все данных для геренации пакетов. Осталось научиться загружать их в наш генератор и использовать\nдля генерации пакетов. Работу продолжим в файле generator.py. Открывайте его\n'

In [18]:
# our_data = pd.read_csv('attack_data/192.168.20.111/generated_result_02.12.2020_05-12-43.scv', sep=",")
# our_data

In [5]:
ddos_data['Flow Pkts/s']

KeyError: ('Fwd Header Len', 'Bwd Header Len')

In [27]:
fd = pd.read_csv("attack_data/25.147.77.13/generated_result_06.12.2020_03-59-40.csv", sep=",")
fd

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Start Timestamp,Flow End Timestamp,Flow Duration,...,Fwd PSH Flags,Bwd PSH Flags,Bwd Pkt Len Max,Bwd Pkts/s,Flow Byts/s,Flow Pkts/s,Down/Up Ratio,Flow IAT Mean,Active Mean,Idle Std
0,192.168.2.109-25.147.77.13-1785-443-6,192.168.2.109,1785,25.147.77.13,443,6,06/12/2020 03:58:58 AM,1.607216e+09,1.607216e+09,96006.0,...,0,0,332,177.072266,32164.656376,249.984376,10.028571,14019.342175,59281.195767,13546.894074
1,192.168.2.109-25.147.77.13-1785-443-6,192.168.2.109,1785,25.147.77.13,443,6,06/12/2020 03:58:58 AM,1.607216e+09,1.607216e+09,146008.0,...,0,0,331,171.223495,27423.154896,239.712892,9.010000,14019.342175,59281.195767,13546.894074
2,192.168.2.109-25.147.77.13-1785-443-6,192.168.2.109,1785,25.147.77.13,443,6,06/12/2020 03:58:59 AM,1.607216e+09,1.607216e+09,91006.0,...,0,0,186,175.812584,23141.331341,241.742303,7.775000,14019.342175,59281.195767,13546.894074
3,192.168.2.109-25.147.77.13-1785-443-6,192.168.2.109,1785,25.147.77.13,443,6,06/12/2020 03:58:59 AM,1.607216e+09,1.607216e+09,16001.0,...,0,0,217,249.984376,41622.398600,312.480470,15.650000,14019.342175,59281.195767,13546.894074
4,192.168.2.109-25.147.77.13-1785-443-6,192.168.2.109,1785,25.147.77.13,443,6,06/12/2020 03:58:59 AM,1.607216e+09,1.607216e+09,29002.0,...,0,0,184,206.882284,26515.412730,275.843045,8.612500,14019.342175,59281.195767,13546.894074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,192.168.2.112-25.147.77.13-2749-443-6,192.168.2.112,2749,25.147.77.13,443,6,06/12/2020 03:58:54 AM,1.607216e+09,1.607216e+09,363021.0,...,0,11,1107,145.997064,112337.853733,234.146234,30.394149,75084.266667,674143.842105,41139.099769
496,192.168.2.112-25.147.77.13-2749-443-6,192.168.2.112,2749,25.147.77.13,443,6,06/12/2020 03:58:54 AM,1.607216e+09,1.607216e+09,397023.0,...,0,8,1252,136.012271,101953.791090,219.130882,29.549434,75084.266667,674143.842105,41139.099769
497,192.168.2.112-25.147.77.13-2749-443-6,192.168.2.112,2749,25.147.77.13,443,6,06/12/2020 03:58:55 AM,1.607216e+09,1.607216e+09,472028.0,...,0,12,1131,146.177769,112533.578517,235.155542,30.209753,75084.266667,674143.842105,41139.099769
498,192.168.2.112-25.147.77.13-2749-443-6,192.168.2.112,2749,25.147.77.13,443,6,06/12/2020 03:58:55 AM,1.607216e+09,1.607216e+09,575033.0,...,0,11,1177,142.600512,101851.545911,229.552043,27.431068,75084.266667,674143.842105,41139.099769
